### Установки

In [3]:
!pip install pymorphy2
%pip install spacy
!python -m spacy download ru_core_news_lg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 71.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13721 sha256=eacba3e7707775b3c44035ab12c7a30d248b48976d969708a0e9abc11a3911d8
  Stored in directory: /root/.cache/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2023-04-07 12:02:21.111535: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
202

### Импорт

In [117]:
from pymorphy2 import MorphAnalyzer
from gensim.models import Word2Vec
from nltk.corpus import stopwords
from random import randint

import pandas as pd
import numpy as np
import pickle
import nltk
import json
import gc
import re



*   распаковка json
*   знакомство с данными
*   разбиение текстов на  предложения
*   лемматизация и очищение от стоп-слов



### распаковка json, приведение к удобному виду для pandas

In [52]:
with open('/content/train.json', 'r', encoding='utf-8') as f:
  json_data = json.load(f)
len(json_data)

FileNotFoundError: ignored

In [53]:
json_data[0]['extracted_part']

NameError: ignored

In [ ]:
json_data[0]['text'][1279:1343]


In [ ]:
lenght = len(json_data)
for i in range(lenght):
  target = json_data[i]['extracted_part']['text'][0]
  if target:
    json_data[i]['target'] = target

  else: 
    json_data[i]['target'] = np.nan

  del json_data[i]['extracted_part']
  
json_data[0]

In [ ]:
with open('train_unpacked.json', 'w') as f:
  json.dump(json_data, f)

In [ ]:
del json_data
gc.collect()

### откроем в pandas посмотрим на данные

In [54]:
df = pd.read_json('/content/train_unpacked.json', encoding='utf-8')
df.sample(3)

,id,text,label,target
1467,134794731,Извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,Размер обеспечения гарантийных обязательств 36...
925,712278751,«УТВЕРЖДАЮ» Директор ГУП СК «Ставрополькоммунэ...,обеспечение исполнения контракта,Заказчик вправе установить в документации о за...
943,598410562,ИНФОРМАЦИОННЫЙ ЛИСТ Реестровый номер - 655 ЗК ...,обеспечение исполнения контракта,Размер обеспечения исполнения договора 5% от Н...


In [55]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1799 entries, 0 to 1798
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1799 non-null   int64 
 1   text    1799 non-null   object
 2   label   1799 non-null   object
 3   target  1492 non-null   object
dtypes: int64(1), object(3)
memory usage: 56.3+ KB


Посмотрим поближе на сами тексты

In [56]:
df['label'].value_counts()

обеспечение исполнения контракта        988
обеспечение гарантийных обязательств    811
Name: label, dtype: int64

In [68]:
string = randint(0, 1799)
df.loc[string, 'text'], df.loc[string, 'label'], df.loc[string, 'target'] 

('8 Проект ГОСУДАРСТВЕННЫЙ КОНТРАКТ №_____________________________ Идентификационный код закупки: ________________________________________ г. Псков «____»____________ 2022 г. Комитет юстиции Псковской области, именуемый в дальнейшем «Заказчик», в лице председателя Комитета Шерстобитова Сергея Валерьевича, действующего емке поставленного Товара в случае выявления несоответствия Товара условиям Контракта, если выявленное несоответствие не препятствует приемке этого Товара и устранено Поставщиком. 6.17. В случае недопоставки и (или) поставки некачественного Товара Заказчик вправе потребовать от Поставщика осуществить замену поставленного некачественного Товара Товаром надлежащего качества, соответствующего условиям Контракта, и (или) потребовать осуществить поставку недостающего Товара в течение 5 (пяти) календарных дней с даты получения претензии Заказчика. 6.18. Возврат поставленного Товара ненадлежащего качества осуществляется за счет средств Поставщика. 6.19. Поставщик, по согласовани

В текстах не везде расставлены знаки препинания, поэтому мы не можем чётко разделить предложения и можем ориентироваться только на регистр

Сложность составляют слова, которые в юридических документах пишутся с заглавной буквы: Заказчик, Договор, Сторон и т.д.

Попробуем написать функцию для разделения текстов на предложения

In [58]:
def lower_the_unnessesary_words(text):
  # сделаем слова, которые пишутся с заглавной буквы и не стоят в начале предложения прописными
  capital_words_exceptions = [
      'Заказчик', 'Получател', 'Договор', 'Поставщик', 
      'Федер', 'Закон', 'Росси', 'Контракт', 'Гражданск', 
      'Министерств', 'Директор', 'Сторон', 'Оборудован', 
      'Акт', 'МВД', 'НМЦК'
  ]
  splitted_text = text.split(' ')
  for i in range(len(splitted_text)):
    for exception in capital_words_exceptions:  
      if type(splitted_text[i]) is str and splitted_text[i].startswith(exception):
        splitted_text[i] = splitted_text[i].lower()

  return ' '.join(splitted_text)
          

def text_separation_by_sentences(text, min_sentence_lenght=20):
  # сначала разделим текст по заглавным и уберём неинформативные предложения
  sentences = []
  text = text.replace('\n', ' ')
  text = text.split('.')
  text = ' '.join(text)

  text = lower_the_unnessesary_words(text)
  raw_sentences = re.split(r'\s+(?=[А-Я])', text)

  for i in raw_sentences:
    if len(i) > min_sentence_lenght:
      sentences.append(i)

  return sentences

Наш разделитель не будет идеально делить предложения, т.к. могут быть предложения, у которых в начале стоит одно из слов-исключений, либо когда мы не внесли слово в список слов-исключений, но оно написано с большой буквы 

### разбиение текстов 

In [69]:
df.sample(4)

,id,text,label,target
1271,279645294,Извещение о проведении электронного аукциона д...,обеспечение гарантийных обязательств,Размер обеспечения гарантийных обязательств 63...
689,793994530,УТВЕРЖДАЮ Главный врач ГАУЗ СО «Новолялинская ...,обеспечение исполнения контракта,Обеспечение исполнения договора 5% от начально...
873,334770469,"УТВЕРЖДАЮ Начальник департамента логистики, ма...",обеспечение исполнения контракта,Заказчик вправе установить в документации о за...
4,211645258,Извещение о проведении электронного аукциона д...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%


попробуем разбить тексты на предложения с помощью spacy



In [17]:
import spacy

In [ ]:
string = randint(0, 1799)
text = df.loc[string, 'text']

In [27]:
nlp = spacy.load('ru_core_news_lg')
tokens = nlp(text)

In [29]:
for sent in tokens.sents:
    print(sent)
    print()


Идентификационный код закупки: 222636720035863670100100320010147323 КОНТРАКТ №____ на поставку продуктов питания город ______________ «___»__________ 20__ год Государственное казенное учреждение Самарской области ''Волжский социально-реабилитационный центр для несовершеннолетних ''Тополек'', именуемое в дальнейшем «заказчик»,  который является казенным учреждением; 2) осуществления закупки услуги по предоставлению кредита; 3) заключения бюджетным учреждением, государственным, муниципальным унитарными предприятиями контракта, предметом которого является выдача независимой гарантии.

Заказчик вправе установить, что настоящий раздел не применяется в случае заключения Контракта по результатам проведения электронного запроса котировок (за исключением случая, предусмотренного подпунктом "б" пункта 2 части 10 статьи 24 Федерального закона «О контрактной системе в сфере закупок товаров, работ, услуг для обеспечения государственных и муниципальных нужд»).

По контрактам, подлежащим казначейско

Наше разбиение работает лучше, т.к. spacy разбивает только по точкам, а в нашей задаче много текстов, где их нет

In [70]:
for i in range(len(df)):
  sentences = text_separation_by_sentences(df.loc[i, 'text'])
  df.at[i, 'text'] = sentences

также spacy для этой задачи работает довольно медленно, т.к. выполняет не только операцию разбиения на предложения, а также использует 

In [71]:
string = randint(0, 1799)
df.loc[string, 'text'], df.loc[string, 'label'], df.loc[string, 'target'] 

(['КОНТРАКТ № на поставку бензина автомобильного',
  'Аи-92 и дизельного топлива для нужд муниципального унитарного предприятия',
  'Рамешковского района «МУПАРР» п',
  'Рамешки «___» __________ 2022 г',
  'Муниципальное унитарное предприятие',
  'Рамешковского района лучае выявления факта не соответствия поставляемого товара сертификату соответствия, паспорту качества завода-изготовителя поставщик возмещает заказчику все понесенные убытки  5 3  поставщик гарантирует, что поставляемый',
  'Товар свободен от прав третьих лиц, не является предметом спора, не находится в залоге, под арестом или иным обременением, а также гарантирует, что к заказчику не будут применены меры материальной ответственности по искам третьих лиц в отношении нарушения патентных прав, а также прав на использование торговой марки или промышленных образцов, связанных с использованием',
  'Товара или любой их части в российской федерации  6',
  'Обеспечение исполнения контракта 6 1',
  'В целях обеспечения исполнения

Алгоритм справляется с разбиением предложений достаточно хорошо, качество разбиения можно повысить, добавляя и убирая слова-исключения. 

Ещё одна проблема заключается в том, что в таргете у нас не всегда стоят законченные предложения, что может сказаться на accuracy. Например:


*   Участник закупки, с которым заключается договор, обязан представить обеспечение исполнения договора в размере 5% от цены

Полное предложение: "*Участник закупки, с которым заключается договор, обязан представить обеспечение исполнения договора в размере 5% от цены, по которой в соответствии с законом заключается контракт  17 1  заказчик вправе установить требование обеспечения гарантийных обязательств в случае установления требований к таким обязательствам в соответствии с частью 4 статьи 33 настоящего федерального закона*"

Но и есть и такие конктрукции в таргете:


*   Обеспечение гарантийных обязательств представляется в размере 127 598,00 руб. , что составляет 2 % от начальной (максимальной) цены Договора, предусмотренной документацией о закупке.
*  Размер обеспечения исполнения контракта, гарантийных обязательств 5% от НМЦК в сумме – 683 (шестьсот восемьдесят три) рубля 33 копейки.




Сохраним результаты разбиения:

In [73]:
df.to_json('./train_with_splitted_texts.json')

### Очистка текста

In [101]:
df = pd.read_json('/content/train_with_splitted_texts.json', encoding='utf-8')
df.head()

,id,text,label,target
0,809436509,[Извещение о проведении открытого конкурса в э...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 6593.2...
1,854885310,[ЗАЯВКИ участника запроса котировок в электрон...,обеспечение исполнения контракта,Поставщик должен предоставить обеспечение испо...
2,4382157,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%
3,184555082,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%
4,211645258,[Извещение о проведении электронного аукциона ...,обеспечение исполнения контракта,Размер обеспечения исполнения контракта 10.00%


Функция для очистки лишних знаков препинания, чисел, а также букв английского алфавита

In [75]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [102]:
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()
def lemmatization(text):
  pattern = "[A-Za-z0-9!#$%&'()*+№,./:;<=>?@[\]^_`{|}~—\"\-]+"
  text = re.sub(pattern, ' ', text)
  tokens = []
  for token in text.split():
      if token and token not in stopwords_ru:
          token = token.strip()
          token = morph.normal_forms(token)[0]
          tokens.append(token)
  return tokens

In [103]:
sentence = df.loc[4, 'text'][0]
sentence

'Извещение о проведении электронного аукциона для закупки №0318100007022000091'

In [104]:
lemmatization(sentence)

['извещение', 'проведение', 'электронный', 'аукцион', 'закупка']

In [105]:
%%time
for i in range(len(df)):
  lemmatized_text = []

  for k in range(len(df.loc[i, 'text'])):
    sentence = df.loc[i, 'text'][k]
    sentence =  lemmatization(sentence)
    lemmatized_text.append(sentence)
  df.at[i, 'text'] = lemmatized_text
   

CPU times: user 1min 10s, sys: 203 ms, total: 1min 10s
Wall time: 1min 10s


In [107]:
df.loc[5, 'text']

[['контракт',
  'проект',
  'приобретение',
  'перчатка',
  'резиновый',
  'хозяйственный',
  'столовая'],
 ['красноярский', 'край', 'номер', 'государственный', 'контракт', 'г'],
 ['красноярск', 'г', 'ньсить', 'порядок', 'случай'],
 ['участник',
  'закупка',
  'который',
  'заключаться',
  'контракт',
  'результат',
  'определение',
  'поставщик',
  'соответствие',
  'п',
  'ч',
  'ст',
  'закон',
  'фз',
  'освобождаться',
  'предоставление',
  'обеспечение',
  'исполнение',
  'контракт',
  'число',
  'учёт',
  'ст',
  'закон',
  'фз',
  'обеспечение',
  'гарантийный',
  'обязательство',
  'случай',
  'предоставление',
  'такой',
  'участник',
  'закупка',
  'информация',
  'содержаться',
  'реестр',
  'контракт',
  'заключить',
  'заказчик',
  'подтверждать',
  'исполнение',
  'такой',
  'участник',
  'течение',
  'три',
  'год',
  'дата',
  'подача',
  'заявка',
  'участие',
  'закупка',
  'три',
  'контракт',
  'исполнить',
  'применение',
  'такой',
  'участник',
  'неустойка',
  

In [108]:
df.to_json('train_lemmatized.json')

In [116]:
contract_enforcement = df[df['label'] == 'обеспечение исполнения контракта']['target'].values
garantee_enforcement = df[df['label'] == 'обеспечение гарантийных обязательств']['target'].values

также сохраним таргеты двух классов для модели classy classification

In [118]:
with open('contract_enforcement.pkl', 'wb') as fp:
  pickle.dump(contract_enforcement, fp)

In [120]:
with open('garantee_enforcement.pkl', 'wb') as fp:
  pickle.dump(garantee_enforcement, fp)

### Word2Vec